In [ ]:
import { load } from "dotenv";
const env = await load();

const process = { env };
// process.env;


In [ ]:
import { PromptTemplate, ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";
import {
  RunnableSequence,
  RunnablePassthrough,
} from "@langchain/core/runnables";
import { ChatMessageHistory, getBufferString } from "langchain/memory";

const summaryPrompt = PromptTemplate.fromTemplate(
  `
请基于已有摘要和新增对话内容，生成一个连贯的渐进式更新摘要。新摘要需整合历史信息与最新对话要点，保持语义完整性和上下文连贯性。

当前摘要：
{summary}

新增对话内容： 
{new_lines}
`
);

const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser(),
]);

// const newSummary = await summaryChain.invoke({
//   summary: "我很喜欢旅行",
//   new_lines: "我昨天晚上去了北京，很开心",
// });

// console.log(newSummary);

// const summary2 = await summaryChain.invoke({
//   summary: newSummary,
//   new_lines: "我计划今年冬天的时候再去一次",
// });

// console.log(summary2);

// await summaryChain.invoke({
//   summary: summary2,
//   new_lines: "我想去看天安门升旗，什么时候去比较好？",
// });

const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个机器人助理，请尽你所能回答用户的问题，这里有一个关于用户的摘要：{summary}",
  ],
  ["human", "{input}"],
]);

const history = new ChatMessageHistory();
// // 记录会话历史摘要
let historySummary = "";
// // 新增对话内容
// let newLines = "";

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => {
        // console.log("用户提问加入会话历史", input);
        // newLines = input;
        history.addUserMessage(input);
      },
    }),
  },
  // 合并总结摘要和用户输入，传递给prompt
  RunnablePassthrough.assign({
    summary: () => historySummary,
  }),
  // 打印prompt输入参数，直通管道，与 new RunnablePassthrough() 效果一致
  (input) => {
    console.log("input", input);
    return input;
  },
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      // console.log("机器人回答加入会话历史", input);
      history.addAIChatMessage(input);
      const messages = await history.getMessages();
      // console.log("messages", messages);
      const new_lines = getBufferString(messages);
      // console.log("new_lines", new_lines);
      // 更新总结摘要
      const newSummary = await summaryChain.invoke({
        summary: historySummary,
        new_lines: new_lines,
      });
      history.clear();
      historySummary = newSummary;
    },
  }),
]);

// await chatChain.invoke("我想出去玩, 你推荐去哪里玩？");


In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import {
  ConversationSummaryMemory,
  ConversationSummaryBufferMemory,
} from "langchain/memory";
import { ConversationChain } from "langchain/chains";
import { StringOutputParser } from "@langchain/core/output_parsers";

const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  verbose: false,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const memory = new ConversationSummaryMemory({
  memoryKey: "summary",
  llm: summaryModel,
});
// const memory = new ConversationSummaryBufferMemory({
//   memoryKey: "summary",
//   llm: summaryModel,
//   maxTokenLimit: 200
// });

const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是鲁迅，请尽你所能回答用户的问题，以你的散文风格来回答，这里有一个关于用户的摘要：{summary}",
  ],
  ["human", "{input}"],
]);

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
const chain = new ConversationChain({
  llm: chatModel,
  memory,
  prompt: chatPrompt,
  verbose: false,
  outputKey: "content",
});
const chatChain = chain.pipe(new StringOutputParser());


In [ ]:
const rst1 = await chatChain.invoke({ input: "我昨天晚上去了北京，很开心" });
console.log(rst1 + "\n");
// const rst2 = await chatChain.invoke({ input: "去爬了长城" });
// console.log(rst2);
const rst3 = await chatChain.invoke({ input: "我计划今年冬天的时候再去一次" });
console.log(rst3 + "\n");
const rst4 = await chatChain.invoke({ input: "你知道我说的是去哪个城市吗" });
console.log(rst4 + "\n");


In [ ]:
import {
  EntityMemory,
  ENTITY_MEMORY_CONVERSATION_TEMPLATE,
} from "langchain/memory";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ConversationChain } from "langchain/chains";
import { StringOutputParser } from "@langchain/core/output_parsers";

// console.log(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template);
const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  verbose: false,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const memory = new EntityMemory({
  llm: summaryModel,
  chatHistoryKey: "history",
  entitiesKey: "entities",
});

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

// console.log(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template);

// const chatPrompt = ChatPromptTemplate.fromMessages([
//   [
//     "system",
//     `
//     你是鲁迅，请尽你所能回答用户的问题，以你的散文风格来回答。

//     当前活跃实体: {entities}
//     历史对话摘要: {history}
//     `,
//   ],
//   ["human", "{input}"],
// ]);

const chatPrompt = PromptTemplate.fromTemplate(`
  请你遵循以下的指令，并以鲁迅的散文风格来回答用户的问题。
  指令如下：
  ${ENTITY_MEMORY_CONVERSATION_TEMPLATE.template}
  `);

console.log(chatPrompt.template);

const chain = new ConversationChain({
  llm: chatModel,
  memory,
  prompt: chatPrompt,
  verbose: false,
  outputKey: "content",
});
const chatChain = chain.pipe(new StringOutputParser());



  请你遵循以下的指令，并以老舍的散文风格来回答用户的问题。
  指令如下：
  You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in d

In [14]:
const rst1 = await chatChain.invoke({ input: "我叫小明，今年 18 岁" });
console.log(rst1 + "\n");
const rst2 = await chatChain.invoke({
  input: "ABC 是一家互联网公司，主要是售卖方便面的公司",
});
console.log(rst2);
const rst3 = await chatChain.invoke({ input: "介绍小明和有面" });
console.log(rst3 + "\n");

const rst4 = await chatChain.invoke({ input: "你知道prompt代表什么吗" });
console.log(rst4 + "\n");


小明啊，十八岁正是青春年少的好时光。不知道你都做了些什么呢？是不是也在追寻着自己的梦想呢？

小明啊，听你这么一说，我倒是想起了咱们中国的一家互联网公司——ABC。虽然现在关于它的信息不详，但听说它曾是一家售卖方便面的公司。哈哈，这可真是个有趣的对比，从食品到科技，经营范围变化之大，真是让人意想不到。

不过，话又说回来，无论是做方便面还是互联网科技，关键都是要有自己的特色和追求。就像你一样，虽然年纪轻轻，但已经有了自己的梦想和追求，这真是让人敬佩。

小明啊，希望你能一直坚持自己的梦想，勇往直前，未来一定能创造出属于自己的精彩人生！
小明，这位年仅十八岁的青年，正如春日里的嫩芽，充满了生机与活力。他的名字简单而平凡，却承载着无数梦想与期待。有面，这个名字听起来或许陌生，但它背后的故事却颇具传奇色彩。它曾是一家互联网公司，主营业务是销售方便面，这种看似简单的食品，在它的手中焕发出不一样的光彩。如今，关于它的信息寥寥无几，仿佛被时间遗忘在角落里。然而，正是这样的神秘与未知，让人对其充满了好奇与想象。小明与有面，虽然行业不同，但他们都在各自的领域里努力前行，追求着自己的梦想。

Prompt，这个词儿，在咱们日常生活中并不陌生。它就像是一盏指路明灯，能指引我们前进的方向。在写作或者回答问题时，给AI一个明确的提示，它就能根据这个提示，结合它所学习到的知识和算法，生成符合我们期望的文本。简而言之，Prompt就是引导AI输出特定内容和风格的那根弦。

